In [43]:
import pandas as pd
import numpy as np
import artm
from tqdm import tqdm_notebook
import time
from proress_bar import log_progress
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 34 ms


In [44]:
with open('logs.txt') as f_in:
    logs = f_in.readlines()

time: 41.6 s


In [45]:
small_logs = logs[::2]

time: 13.7 s


In [52]:
small_logs = small_logs[::2]

time: 89.5 ms


In [53]:
len(small_logs)

1243423

time: 9.44 ms


In [54]:
with open('small_log.txt', 'w') as f_out:
    for i in log_progress(small_logs):
        f_out.write(i)

time: 1.22 s


In [55]:
bach_vectorizer = artm.BatchVectorizer(data_path='small_log.txt',
                                      data_format='vowpal_wabbit', target_folder='batch_small',
                                      batch_size=20)

time: 42 s


In [56]:
T = 30   # количество тем
topic_names=["sbj"+str(i) for i in range(T-1)]+["bcg"]

model_artm = artm.ARTM(num_topics=T, topic_names=topic_names, reuse_theta=True,
                       num_document_passes=1)

time: 2.83 s


In [57]:
np.random.seed(1)
dictionary = artm.Dictionary()
log_progress(dictionary.gather(data_path=bach_vectorizer.data_path))
log_progress(model_artm.initialize(dictionary))

<generator object log_progress at 0x1166594f8>

time: 12.6 s


In [58]:
model_artm.scores.add(artm.TopTokensScore(name='metric1', num_tokens=15))

time: 1.92 ms


In [ ]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='smoothing', dictionary=dictionary,
                                                         topic_names='bcg', tau=1e5))

time: 1.31 ms


In [ ]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=30)

In [ ]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='stimulates',
                                                            dictionary=dictionary,
                                                            topic_names=["sbj"+str(i) for i in range(0, 29)], tau=-1e5))

In [ ]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=15)

In [ ]:
for topic_name in model_artm.topic_names:
    print (topic_name + ':')
    for word in model_artm.score_tracker["metric1"].last_tokens[topic_name]:
        print(word, end=' ')
    print()